In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Mar 20 23:58:44 2020

@author: thomas emmert
"""

import pandas as pd
from datetime import datetime
import cufflinks
cufflinks.go_offline(connected=True)
#cufflinks.init_notebook_mode(connected=True)

countryList = ["Japan","Germany","Italy","Austria","Spain","Republic of Korea","Sweden","France","United States of America","United Kingdom"]

#path_to_csv = '/home/thomas/git/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
path_to_csv = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
# https://population.un.org/wpp/Download/Standard/Population/
#path_to_pop = '/home/thomas/git/WPP2019_POP_F01_1_TOTAL_POPULATION_BOTH_SEXES.xlsx'
path_to_pop = 'https://population.un.org/wpp/Download/Files/1_Indicators%20(Standard)/EXCEL_FILES/1_Population/WPP2019_POP_F01_1_TOTAL_POPULATION_BOTH_SEXES.xlsx'

dataset_raw = pd.read_csv(path_to_csv)
datapop_raw = pd.read_excel(io=path_to_pop, sheet_name='ESTIMATES',header=16)

datapop = datapop_raw.loc[datapop_raw['Type']=='Country/Area']
datapop.index = datapop['Region, subregion, country or area *'].tolist()
datapop = datapop.loc[:,['2020']]
country_list_pop = datapop.index.tolist()

dataset = dataset_raw
dataset.index = dataset['Country/Region'].tolist()
dataset = dataset.groupby(dataset.index).sum()
dataset = dataset.drop(["Lat","Long"], axis=1)

# Map the country names of dataset to the official UN names, where not matching
country_list_mapping ={'Bolivia':"Bolivia (Plurinational State of)",
 'Brunei':'Brunei Darussalam',
 'Burma':'Myanmar',
 'Congo (Brazzaville)':'Congo',
 'Congo (Kinshasa)':"Democratic Republic of the Congo",
 "Cote d'Ivoire":"Côte d'Ivoire",
 'Diamond Princess':'Diamond Princess', # Missing: ship
 'Iran':"Iran (Islamic Republic of)",
 'Korea, South':"Republic of Korea",
 'Kosovo':'Kosovo', # Missing: partially recognized country
 'Laos':"Lao People's Democratic Republic",
 'MS Zaandam':'MS Zaandam', # Missing: ship
 'Moldova':"Republic of Moldova",
 'Russia':'Russian Federation',
 'Syria':"Syrian Arab Republic",
 'Taiwan*':"China, Taiwan Province of China",
 'Tanzania':"United Republic of Tanzania",
 'US':"United States of America",
 'Venezuela':"Venezuela (Bolivarian Republic of)",
 'Vietnam':"Viet Nam",
 'West Bank and Gaza':"State of Palestine"}

dataset.rename(index=country_list_mapping,inplace=True)
country_list_data = dataset.index.tolist()


dataset_transp = dataset.transpose()
datestr = dataset_transp.index.tolist()
datetime_object=[]
for dstr in datestr:
    datetime_object.append(datetime.strptime(dstr, '%m/%d/%y'))

dataset_transp.index = datetime_object

datasetred = dataset_transp.loc[:,country_list_data]
datapop_red = datapop.reindex(country_list_data)

dataset_norm = datasetred/ datapop_red.squeeze() /1000

fig = dataset_norm.figure()
data = fig['data']
for dat in data:
    if not(dat.name in countryList):
        dat.visible='legendonly'
    
dataset_norm.iplot(data=data,title="Infizierte pro Einwohner", filename="my_barplot" ,rangeslider=True, asFigure=True)